# LL84 Processing

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
pd.set_option('display.max_columns', None)

## Step 0: Load in raw data

In [ ]:
#load in raw data
# TODO: make sure we get the columns related to on-site FF infra

ll_raw = pd.read_csv('../data/raw_data/DOB/LL84 Energy Efficiency/NYC_Building_Energy_and_Water_Data_Disclosure_for_Local_Law_84__2022-Present__20250915.csv')
schools = gpd.read_file('../data/processed_data/school_points_with_lcgms.geojson')

## Step 1: Match schools to LL84 data using building codes

In [ ]:
#go through and match buildings based on codes first
matches = []
for bc in schools['Building Code']:
    #find matches using building codes
    df = ll_raw[ll_raw['Property Name'].str.contains(bc)].copy()
    #drop buildings that are not schools, daycare centers, or educational facilities, or worship facilities but that might occupy the same lot
    df = df[df['List of All Property Use Types (GFA) (ft²)'].str.contains('|'.join(['K-12 School', 'Pre-school/Daycare', 'Other - Education', 'Worship Facility']), regex=True)]

    #if no matches are found, try to use BBL to match
    if df.shape[0] == 0:
        try:
            addr = schools[schools['Building Code'] == bc]['Borough Block Lot'].values[0].casefold()
        except:
            addr = None
        df = ll_raw[ll_raw['NYC Borough, Block and Lot (BBL)'].str.casefold() == addr].copy()

    #once aggain, drop buildings that are not schools, daycare centers, or educational facilities, but that might occupy the same lot
    df = df[df['List of All Property Use Types (GFA) (ft²)'].str.contains('|'.join(['K-12 School', 'Pre-school/Daycare', 'Other - Education', 'Worship Facility']), regex=True)]
    
    #if multiple entries are found, select the most recent ones
    if df.shape[0] > 1:
        maxyear = df['Calendar Year'].unique().max().item()
        df = df[df['Calendar Year'] == maxyear]
    
        #for matches with multiple buildings, drop any at alternate addresses
        """
        if df.shape[0] > 1:
            #if address does not exist in school table, will return None
            try:
                addr = schools[schools['Building Code'] == bc]['Primary Address'].values[0].casefold()
            except:
                addr = None
            df = df[df['Address 1'].str.casefold() == addr]
            try:
                addr = schools[schools['Building Code'] == bc]['Borough Block Lot'].values[0].casefold()
            except:
                addr = None
            df = df[df['NYC Borough, Block and Lot (BBL)'].str.casefold() == addr].copy()
        #if there are still multiple entries, choose the one that corresponds to a single building
        """
        if df.shape[0] > 1:
            df = df[~df['NYC Building Identification Number (BIN)'].str.contains(',')]

    #if there are still multiple matched buildings, skip the school
    if df.shape[0] > 1:
        df = df[[]]        

    
    df['Building Code'] = bc
    matches.append(df)


ll84_matched_bc = pd.concat(matches)

### Manual matching to correct for missed matches

In [ ]:
# the first four entries here correspond to schools that were initially filtered out of matches (generally due to multiple buildings), but still had a valid match in the ll84 database, whereas the rest correspond to schools with multiple matched buildings
manual_matches = {
    'K241': 36863,
    'Q774': 47929,
    'M838': 5179,
    'X445': 36862,
    'K286' : 52066,
    'K092' : 38626,
    'K490' : 52758,
    'M170' : 49040,
    'MCDT' : 46251,
    'M908' : 34331,
    'K535' : 52810,
    'K312' : 36864,
    'Q136' : 36867,
    'XCKQ' : 57812,
    'X862' : 51985,
    'X063' : 51984,
    'X008' : 52538,
    'X122' : 52376,
    'X826' : 39016
    
}


manual_add = ll_raw.loc[manual_matches.values()]
manual_add['Building Code'] = manual_matches.keys()

ll84_matched_bc = pd.concat([ll84_matched_bc, manual_add])

## Step 3: Merge LL84 dataframe with the schools dataframe

We choose a subset of columns to retain in the merged dataframe. These can be changed by editing `data columns` below.

In [ ]:
data_columns = [
           'ENERGY STAR Score',
           'Site EUI (kBtu/ft²)',
           'Site Energy Use (kBtu)',
           'Percent Electricity',
           'Direct GHG Emissions (Metric Tons CO2e)',
           'Direct GHG Emissions Intensity (kgCO2e/ft²)',
           'Water Use (All Water Sources) (kgal)']

ll84_matched_bc[data_columns] = ll84_matched_bc[data_columns].apply(pd.to_numeric, errors='coerce')


merged = schools.merge(ll84_matched_bc[['Building Code'] + data_columns]
                       .drop_duplicates(subset='Building Code', keep='first'), on="Building Code", how='left')

In [ ]:
#Here, we shorten column names for the final shapefile

#pre-existing shortened names
short_col_map = {'Location Name': 'Loc_Name',
    'Managed By Name': 'Managed_By',
    'Location Code': 'Loc_Code',
    'Building Code': 'Bldg_Code',
    'ATS': 'ATS',
    'Primary Address': 'Address',
    'City': 'City',
    'State': 'State',
    'Zip': 'Zip',
    'Borough Block Lot': 'BBL',
    'Census Tract': 'C_Tract',
    'Community District': 'Comm_Dist',
    'Council District': 'Council_Di',
    'geometry': 'geometry',
    'BEDS Number': 'BEDS_Num',
    'Location Type Description': 'Loc_Type_D',
    'Location Category Description': 'Loc_Cat_D',
    'Grades': 'Grades',
    'Grades Final': 'Grades_Fin',
    'Open Date': 'Open_Date',
    'NTA': 'NTA',
    'NTA_Name': 'NTA_Name',
    'Principal Name': 'Princ_Name',
    'Principal Title': 'Princ_Titl',
    'Principal Phone Number': 'Princ_Phon',
    'Fax Number': 'Fax_Num',
    'Geographical District Code': 'GeoDisCode',
    'Administrative District Code': 'AdDistCode',
    'Administrative District Location Code': 'AdDistLocC',
    'Administrative District Name': 'AdDistName',
    'Community School Sup Name': 'ComScSupNa',
    'BCO Location Code': 'BCOLocCode',
    'in_LCGMS': 'in_LCGMS',
    'full_address': 'full_addr',
    'google_location_type': 'g_loc_type',
    'lat': 'lat',
    'lng': 'lng',
    'distance_to_nearest_peaker_miles': 'peaker_mi'
}

#shortened names for our columns
short_col_map = short_col_map | {
    'ENERGY STAR Score' : 'eng_star',
    'Site EUI (kBtu/ft²)' : 'eui_norm',
    'Site Energy Use (kBtu)': 'eui_raw',
    'Percent Electricity': 'pct_elec',
    'Direct GHG Emissions (Metric Tons CO2e)' : 'ghg_raw',
    'Direct GHG Emissions Intensity (kgCO2e/ft²)' : 'ghg_norm',
    'Water Use (All Water Sources) (kgal)' : 'water_use'}


In [ ]:
short_col_map

In [ ]:
#A couple kBTU values may get dropped due to errors in kBTU calculation in original dataset
merged.rename(columns=short_col_map).to_file(
    '../data/processed_data/energy_water/school_points_with_lcgms_ll84.shp',
      )

## JW's Alternative Attempt

Join on Extracted Building Code where possible

In [ ]:
# Step zero: filter for K-12 schools
ll84_edu = ll_raw[ll_raw['List of All Property Use Types (GFA) (ft²)'].fillna('').str.contains('|'.join(['K-12 School', 'Pre-school/Daycare', 'Other - Education', 'Worship Facility']), regex=True)]

# Step one: join records building code extracted from regex to "Property Name" field
extracted_building_codes = ll84_edu['Property Name'].str.extractall(r'([LKMQRX]\d{3})')
# For property names with multiple building codes, expand the dataframe so each building code has its own row
ll84_edu_bc = extracted_building_codes.droplevel('match').join(ll84_edu).rename(columns={0:'Building Code'})
# Keep only the most recent record for each extracted building code/property ID combination (need both so we can see where there are multiple properties tied to a single building code)
# primarily_k12_bc = primarily_k12_bc.sort_values('Report Submission Date').drop_duplicates(subset=['Property ID', 'Building Code'], keep='last')
ll84_edu_bc = ll84_edu_bc.sort_values('Report Submission Date').drop_duplicates(subset=['Building Code'], keep='last')
schools_matched_on_bc = schools[['Location Name', 'Location Code', 'Building Code', 'Borough Block Lot']].merge(ll84_edu_bc, on='Building Code', how='left', indicator=True).rename(columns={'_merge':'_merge_bc'})
# TODO: The above merge results in a small join explosion for building codes that are associated with multiple properties in the LL84 data. Need to investigate further.


Join on Extracted BBL where possible

In [ ]:
# Step two: for remaining un-joined records, attempt join on BBL
unmatched_bc_schools_loc_codes = schools_matched_on_bc[schools_matched_on_bc['_merge_bc']!='both']['Location Code'].unique()
unmatched_bc_schools = schools[schools['Location Code'].isin(unmatched_bc_schools_loc_codes)].copy()

# NOTE: @Abhi this is where I diverged just a little from your approach. Maybe we can add in to your code?
# LL84 sometimes lists multiple BBLs for a single property. Here, I explode those into one row per BBL to increase chances of a match on BBL.
ll84_edu_bbl = pd.DataFrame(ll84_edu['NYC Borough, Block and Lot (BBL)'].str.split(';', expand=True).stack().droplevel(1)).join(ll84_edu).rename(columns={0:'BBL'})
# Some BBLs have hyphens, so removing those
ll84_edu_bbl['BBL'] = ll84_edu_bbl['BBL'].str.replace('-', '')
# Drop duplicated BBLs; keep most recent record
# ll84_edu_bbl_bbl = ll84_edu_bbl.sort_values('Report Submission Date').drop_duplicates(subset=['Property ID', 'NYC Borough, Block and Lot (BBL)'], keep='last')
ll84_edu_bbl = ll84_edu_bbl.sort_values('Report Submission Date').drop_duplicates(subset=['BBL'], keep='last')
schools_matched_on_bbl = unmatched_bc_schools.merge(ll84_edu_bbl, left_on='Borough Block Lot', right_on='BBL', how='left', indicator=True).rename(columns={'_merge':'_merge_bbl'})

# Check if duplicate matches are from different Property IDs or same building at different times
schools_matched_on_bbl[schools_matched_on_bbl['Location Code'].duplicated(keep=False)].groupby('Location Code').agg({
    'Property ID': 'nunique',
    'Report Submission Date': 'nunique'
}).rename(columns={'Property ID': 'num_properties', 'Report Submission Date': 'num_dates'})

Combine Building Code and BBL matches

In [ ]:
# Combine different join results into single DF
ll84_schools_combined = pd.concat([
    schools_matched_on_bc[schools_matched_on_bc['_merge_bc']=='both'],
    schools_matched_on_bbl[schools_matched_on_bbl['_merge_bbl']=='both']
], ignore_index=True)

Inspect differences

In [ ]:
print("unique Building Codes matched from original approach: ", ll84_schools_combined['Building Code'].nunique())
print("unique Building Codes matched from JW approach: ", ll84_matched_bc['Building Code'].nunique())

In [ ]:
ll84_schools_combined[~ll84_schools_combined['Building Code'].isin(ll84_matched_bc['Building Code'].unique())]

In [ ]:
ll84_matched_bc[~ll84_matched_bc['Building Code'].isin(ll84_schools_combined['Building Code'].unique())]